In [ ]:
import pandas as pd
import numpy as np
import scipy as sp
import plotly.offline as plotly
import plotly.figure_factory as ff
import plotly.graph_objs as go
import plotly.io as pio
from plotly import tools
import cufflinks as cf
import envelope_plot as ep
from ipywidgets import interact, interactive
from IPython.display import display
import ipywidgets as widgets
import os
import math

cf.go_offline()
plotly.offline.init_notebook_mode(connected=True)

def extract_keys(my_dict, key_list):
    return {k: my_dict[k] for k in key_list}
    

# Prepare data

Every domain instance requires the optimal path length which is
read in via a json results file in the same format as the experiment results.

Specify the paths to the results yielding information on the optimal plans below.
Then specify paths to the experiment results file. A dataframe will be constructed
using this information


In [ ]:
# optimal_plans = ['../results/astar_minima1500.json','../results/astar_orz100d.json',
#                  '../results/astar_korf4real.json','../results/astar_hansen-bigger-d-wide3_no-startup.json']
optimal_plans = [
    '../results/dataFRESH_BENCHMARK_OPTIMAL-09-35-20-07-19.json',
#     '../results/dataRACE_OPTIMAL-19-01-15-08-19.json.gz',
    '../results/astar_korf4real.json'
]

experiments = [
#     '../results/dataTBA_LSS-14-05-03-08-19.json.gz',
#     '../results/dataRR_BI-ES-11-41-04-08-19.json.gz',
#     '../results/dataBACKWARD_BI-ES-16-03-04-08-19.json.gz',
#     '../results/dataRACE_BI1-16-01-11-08-19.json.gz',
#     '../results/dataRACE_BI2-22-29-12-08-19.json.gz',
#     '../results/dataRACE_TBALSS-11-23-10-08-19.json.gz',
    '../results/dataDRAFT-20-12-15-08-19.json.gz',
    '../results/dataDRAFT_BACK-19-12-16-08-19.json.gz'
]

experiment_data = ep.analyze_results(optimal_plans, experiments)

# experiment_data[
#     (experiment_data['domainPath'].str.contains('gridmap/room-map/8room', na=False)) &
#     (experiment_data['algorithmName'] == 'BI_ES') & 
#     (experiment_data['actionDuration'] == 3000) &
#     (experiment_data['lookaheadStrategy'] == 'GBFS')
# ].to_json('8room_res.json', orient='index')

# experiment_data['withinOpt']


In [ ]:
# Within Optimal plot data prep

domains = {
    'Minima 1500x1500':'minima1500',
    'Uniform 1500x1500':'uniform1500',
    'Dragon Age: Origins orz100d (Grid World)':'gridmap/dao-map/orz100d',
    'Dragon Age: Origins ost000a (Grid World)':'gridmap/dao-map/ost000a',
    'Rooms: 8 (Grid World)':'gridmap/room-map/8room',
    'Rooms: 16 (Grid World)':'gridmap/room-map/16room',
    'Starcraft: The Frozen Sea (Grid World)':'gridmap/sc1-map/TheFrozenSea',
    'Starcraft: Cauldron (Grid World)':'gridmap/sc1-map/Cauldron',
    'Sliding Tile - Korf 4x4':'tiles/korf/4/real',
    'Racetrack (Hansen, 3-wide)': 'hansen-bigger-d-wide3'
#     'Dragon Age: Origins orz (Racetrack)':'racetrack/dao-map/orz100d',
#     'Dragon Age: Origins ost (Racetrack)':'racetrack/dao-map/ost000a',
#     'Starcraft: The Frozen Sea (Racetrack)':'racetrack/sc1-map/TheFrozenSea',
#     'Starcraft: Cauldron (Racetrack)':'racetrack/sc1-map/Cauldron'
}

master_config = {
    'weight': 'W',
    'lookaheadStrategy': 'Look',
    'envelopeSearchStrategy': 'Env-L',
    'bidirectionalSearchStrategy': 'Bi'
}

just_weight = extract_keys(master_config, ['weight'])
wLookaheadConfig = extract_keys(master_config, ['weight', 'lookaheadStrategy'])
biEsConfig = extract_keys(master_config, [
    'weight', 'lookaheadStrategy', 'envelopeSearchStrategy', 'bidirectionalSearchStrategy'
])

groupings = {
    'LSS_LRTA_STAR':{},
    'BI_ES': biEsConfig,
    'TIME_BOUNDED_A_STAR': wLookaheadConfig
}

bi_lookahead_options = {
    'Greedy Env, Greedy Frontier':0,
    'Greedy Env, A* Frontier': 1,
    'A* Env, Greedy Frontier': 2,
    'A* Env, A* Frontier': 3,
    'Greedy Env, Pseudo F Frontier': 4,
    'A* Env, Psuedo F Frontier': 5
}

expansion_data, cpu_data = ep.prepare_within_opt_plots(experiment_data,domains.values(), groupings,
                                                       percentile=99)

# Plot data

Please use the drop down menus to select the domain instance and the plot type

In [ ]:
def plot_within_opt(domain, display_type, result_type,
                    algorithms, weights, bi_lookahead, biSearchStrategies,
                    xaxis_name, yaxis_name):
    
    # Don't modify original, so copy the data
    data_copy = None
    xaxis_config = None
    if result_type == 'CPU':
        data_copy = cpu_data.get(domain,{}).copy()
        xaxis_config = cpu_data['xaxis']
    elif result_type == 'Expansion':
        data_copy = expansion_data.get(domain,{}).copy()
        xaxis_config = expansion_data['xaxis']
    
    if len(algorithms) == 0 or len(data_copy) == 0:
        return None, None, None
    
    
    trimmed_df = pd.DataFrame({}, columns=data_copy.columns)
    
    for alg in algorithms:
        alg_data = data_copy[data_copy.algCode == alg]
        
        # Filters
    
        # Generic filter loop - if param exists and values are specified, filter by it
    
        alg_filters = groupings[alg]
        test_weights = weights
        if 'Greedy' in weights:
            test_weights = test_weights + (1.0,)
        for group, values in zip(
            ['weight', 'bidirectionalSearchStrategy'],
            [test_weights, biSearchStrategies]
        ):
            if group in alg_filters and len(values) > 0:
                alg_data = alg_data[alg_data[group].isnull() | alg_data[group].isin(values)]
                
        # Dealing with Greedy param
        
        if alg == 'TIME_BOUNDED_A_STAR':
            if 'Greedy' not in weights:
                alg_data = alg_data[~(alg_data.lookaheadStrategy == 'GBFS')]
            if 1.0 not in weights:
                alg_data = alg_data[
                    ~((alg_data.lookaheadStrategy == 'A_STAR') & 
                     (alg_data.weight == 1.0))
                ]
            
        if alg == 'BI_ES':
            if 1.0 not in weights:
                alg_data = alg_data[~(
                    (alg_data.weight == 1.0) &
                    ~(
                        (alg_data.lookaheadStrategy == 'GBFS') &
                        (alg_data.envelopeSearchStrategy == 'GBFS')
                    )
                )]
            

            
        # Special Bi-ES: filter by lookahead strategy and generate predecessors
        if alg == 'BI_ES':
            if 0 not in bi_lookahead:
                alg_data = alg_data[~((alg_data.lookaheadStrategy == 'GBFS') &
                                   (alg_data.envelopeSearchStrategy == 'GBFS'))]
            if 1 not in bi_lookahead:
                alg_data = alg_data[~((alg_data.lookaheadStrategy == 'A_STAR') &
                                   (alg_data.envelopeSearchStrategy == 'GBFS'))]
            if 2 not in bi_lookahead:
                alg_data = alg_data[~((alg_data.lookaheadStrategy == 'GBFS') &
                                   (alg_data.envelopeSearchStrategy == 'A_STAR'))]
            if 3 not in bi_lookahead:
                alg_data = alg_data[~((alg_data.lookaheadStrategy == 'A_STAR') &
                                   (alg_data.envelopeSearchStrategy == 'A_STAR'))]
            if 4 not in bi_lookahead:
                alg_data = alg_data[~((alg_data.lookaheadStrategy == 'PSEUDO_F') &
                                   (alg_data.envelopeSearchStrategy == 'GBFS'))]
            if 5 not in bi_lookahead:
                alg_data = alg_data[~((alg_data.lookaheadStrategy == 'PSEUDO_F') &
                                   (alg_data.envelopeSearchStrategy == 'A_STAR'))]
            
        
        trimmed_df = trimmed_df.append(alg_data)
        
        
    if display_type == 'Table':
        table = ff.create_table(trimmed_df)
        table.layout.update({'title':domain})
        
        return (table, None, None)
    elif display_type == 'Plot':
        # cufflinks doesn't support error bars...? At least I can't find it
        # creating plotly plot by hand instead... ugh!
        
        palette = cf.colors.get_scales('ggplot', trimmed_df.algorithmName.nunique())
        num_colors = len(palette)
        data_comp = []
        for idx, alg_name in enumerate(trimmed_df.algorithmName.unique()):
            alg_df = trimmed_df[trimmed_df.algorithmName == alg_name]
            # get error bounds
            error_y = None
            error_x = None
            if 'yLbound' in alg_df.columns:
                error_y = dict(
                    type='data',
                    symmetric=False,
                    array=alg_df.yLbound,
                    arrayminus=alg_df.yRbound,
                    visible=True
                )
            if 'xLbound' in alg_df.columns:
                error_x = dict(
                    type='data',
                    symmetric=False,
                    array=alg_df.xLbound,
                    arrayminus=alg_df.xRbound,
                    visible=True
                )
                
            
            data_comp.append(go.Scatter(
                x = alg_df[xaxis_config['data']],
                y = alg_df.withinOpt,
                xaxis = xaxis_name,
                yaxis = yaxis_name,
                name = alg_name,
                mode = 'lines+markers',
                error_y = error_y,
                error_x = error_x,
                line = dict(
                    color=palette[idx % num_colors]
                )
            ))
        
        xaxis_layout = dict(
            title=xaxis_config['title'],
            type='log',
            anchor=yaxis_name
        )
        yaxis_layout = dict(
            title='Factor of Optimal',
            type='log',
            anchor=xaxis_name
        )
        
        return data_comp, xaxis_layout, yaxis_layout
    else:
        return (None, None, None)


def interactive_plot(domain, display_type, result_type, algorithms, weights, bi_lookahead, biSearchStrategies):
    data_comp, xaxis, yaxis = plot_within_opt(domain, display_type, result_type, algorithms,
                               weights, bi_lookahead, biSearchStrategies,
                               'x', 'y')
    
    if data_comp != None:
        fig_comp = go.Figure(data=data_comp, layout=go.Layout(title=domain, xaxis=xaxis, yaxis=yaxis))
        plotly.iplot(fig_comp, filename='jupyter-within-opt-plot')
    
    
    
weights = list(filter(lambda x: x==x, experiment_data.weight.unique())) # remove NaN
weights.append('Greedy')
plot_widget = interactive(
    interactive_plot,
    domain=domains,
    display_type=['Plot','Table'],
    algorithms=widgets.SelectMultiple(
        options={
            'TBA*':'TIME_BOUNDED_A_STAR',
            'LSS-LRTA*':'LSS_LRTA_STAR',
            'Bidirectional Envelope': 'BI_ES'
        },
        description='Algorithms (Select multiple)',
        value=[]
    ),
    result_type=['CPU','Expansion'],
    weights=widgets.SelectMultiple(
        options=weights, # remove NaN
        description='Weights (Select multiple, or none for all)',
        value=[]
    ),
    bi_lookahead=widgets.SelectMultiple(
        options=bi_lookahead_options,
        description='Bi-ES lookahead strategies',
        value=[2]
    ),
    biSearchStrategies=widgets.SelectMultiple(
        options=experiment_data.bidirectionalSearchStrategy.unique(),
        description='Bi-directional search strategies',
        value=['ROUND_ROBIN']
    )
)

display(plot_widget)


# Button code
# Saves a plot of the current configuration for each domain

handler_output = widgets.Output()
def click_handler(btn):
    with handler_output: # for debug printing
        args = plot_widget.kwargs.copy()
        
        for idx, key_value in enumerate(domains.items()):
            title, domain = key_value
            args['domain'] = domain
            args['xaxis_name'] = 'x'
            args['yaxis_name'] = 'y'
            
            data, xaxis, yaxis = plot_within_opt(**args)
            
            if data is None:
                continue
            fig_comp = go.Figure(data=data, layout=go.Layout(title=title, xaxis=xaxis, yaxis=yaxis))
            pio.write_image(fig_comp, '../output/' + domain.replace('/','_') + '-plot.pdf')
           
            
        
button = widgets.Button(
    tooltip='Plot all domains using the current configuration',
    description='Plot All Domains',
    disabled=False,
    button_style='success'
)
button.on_click(click_handler)

display(button, handler_output)
        

In [ ]:
test = experiment_data[(experiment_data['algorithmName'] == 'BI_ES') &
                       (experiment_data['domainPath'].str.contains('ost')) &
                       (experiment_data['bidirectionalSearchStrategy'] == 'BACKWARD') &
                       (experiment_data['envelopeSearchStrategy'] == 'A_STAR') &
                       (experiment_data['actionDuration'] == 10)
                      ]

print(test.numForwardResets.mean())
print(test.pathLength.mean())
print(test.envelopeResets.mean())

# print(cf.colors.scales())